<a href="https://colab.research.google.com/github/abijith007/Bosch/blob/main/Black_box_Cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import tensorflow as tf
import keras
import numpy  as np
import pandas as pd
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate
from sklearn.metrics import f1_score, accuracy_score, classification_report
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random
import cv2
from tqdm.notebook import tqdm

In [ ]:
# Load Data and training

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes = 10)
y_test = keras.utils.to_categorical(y_test, num_classes = 10)


In [ ]:
model = tf.keras.models.load_model("cifar.h5")
print(model.summary())

In [ ]:
pred_test = model.predict(x_test)

pred_test = np.argmax(pred_test, axis=-1)
pred_test = keras.utils.to_categorical(pred_test , num_classes = 10)

acc = accuracy_score(y_test, pred_test)

pred_test = np.argmax(pred_test, axis=-1)
y_ori = np.argmax(y_test, axis=-1)

print("Test accuracy score : %s "% acc)
print(classification_report(y_ori, pred_test))

In [ ]:
def normalize_attack(X_at,y_at):
  y_at_max = np.argmax(y_at,axis=-1)
  a = y_at_max.tolist()
  unq =np.unique(y_at_max)
  # print(unq)
  k =0 
  minimum =list()
  for i in range(len(unq)-1):
      # print(unq[i], a.count(unq[i]))
      if a.count(unq[i]) !=0:
        # if k == 0:
        #   minimum = list(a.count(unq[i]))
        #   k=k+1
        # else:
        minimum.append(a.count(unq[i]))
  minimum.sort()
  # print(minimum)
  
  sec_min = minimum[1]

  for i in range(len(unq)):
    if i == 0 :
      y_normal =  y_at[np.where(y_at_max==unq[i])[0]][0:sec_min]
      X_normal = X_at[np.where(y_at_max==unq[i])[0]][0:sec_min]
    else:

  
      y_normal = np.concatenate((y_normal,y_at[np.where(y_at_max==unq[i])[0]][0:sec_min]),axis=0)
      X_normal = np.concatenate((X_normal,X_at[np.where(y_at_max==unq[i])[0]][0:sec_min]),axis=0)

  return X_normal,y_normal

In [ ]:
# Inception = tf.compat.v1.keras.applications.inception_v3.InceptionV3()
# features_list = [layer.output for layer in Inception.layers]

In [ ]:
noises= ["gauss","s&p","speckle","normal","bernolli","poisson"]
noise_exclusive = ["gauss","s&p","speckle","normal","bernolli"]
# print(features_list)

In [ ]:
def evalu_quar(out) :   
    a = out.tolist()
    unq =np.unique(out)
    print(unq)
    for i in range(len(unq)):
        print(unq[i], a.count(unq[i]))

In [ ]:
# np.random.seed(12)
# layers = Inception.get_layer('conv2d_'+str(1_1)).get_weights()

# print(layers[0].shape)

# np.save("layer_o.npy",layers[0])

In [ ]:
np.random.seed(12)
# layers = Inception.get_layer('conv2d_'+str(1_1)).get_weights()
def fil():
    
    x1w = np.load("layer_o.npy")
    np.random.shuffle(x1w)
    img = [0,0,0,0,0]
    k= 0
    for i in range(5):
        img[i] = x1w[:,:,:,k]
        for j in range(4):
            l= k+j
            img[i] = np.concatenate((img[i], x1w[:,:,:,l]), axis=0)
        k =k+5
    img_final = img[0]
    for u in range(1,5):
        img_final = np.concatenate((img_final, img[u]), axis=1)
    return img_final


In [ ]:
np.random.seed(12)

def noisy(noise_typ,image):
    row,col= image.shape    
    
    if noise_typ == "gauss":
        row,col= image.shape
        mean = np.random.rand()-0.5
        var = np.random.rand()
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col))
        gauss = gauss.reshape(row,col)
        noisy = image + gauss
        return noisy 
    elif noise_typ == "normal":
        np.random.seed(1234)
        mean = np.random.rand()-0.5
        var = np.random.rand()
        sigma = var**0.5
        samples=np.random.lognormal(mean,sigma,(row,col))
        samples=samples.reshape(row,col)
        noisy = image + samples
        return noisy
    
    elif noise_typ == "bernolli":
        row,col= image.shape
        samples=np.random.binomial(10,0.5,(row,col))
        samples=samples.reshape(row,col)
        noisy = image + samples
        return noisy

    elif noise_typ == "s&p":
        row,col = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        # vals = len(np.unique(image))
        # vals = 2 ** np.ceil(np.log2(vals))
        # noisy = np.random.poisson(image * vals)
        PEAK=1
        noisy = np.random.poisson(image / 255.0 * PEAK) / PEAK * 255 
        
        return noisy
    elif noise_typ =="speckle":

        gauss = np.random.randn(row,col)
        gauss = gauss.reshape(row,col)        
        noisy = image + image * gauss
        return noisy


In [ ]:
np.random.seed(12)

def random_square(img,x,y,max_height_x,max_height_y ):

    box = np.array([np.random.randint(0,255)]*max_height_x*max_height_y).reshape(max_height_x,max_height_y)
    try: 
        
        img[x:x+max_height_x, y:y+max_height_y] = box
    except:
        pass
    return img

In [ ]:
np.random.seed(12)

def query(number_attack_vector):
    FLAG =0
    attack_data = np.random.random((100000,32,32,3))

    y_pred = []
    dim = 32
    l =np.random.randint(0,10)
    image_noise = np.random.randint(0,5)
    k =0
    for t in tqdm(range(number_attack_vector)):
     
        
      if image_noise <2:
        image = np.random.random((dim,dim,3))*0+255
      elif image_noise == 2:
        image = np.random.random((dim,dim,3))*0
      else:
        image = fil()
        image = cv2.resize(image, (dim,dim),interpolation = cv2.INTER_NEAREST) 
        image = image[:,:,0:3]
       
      if l == 1:
        for j in range(random.randint(1,20)):

          if image_noise >1:
            type_of_noise = random.randint(0,len(noise_exclusive)-1)
          else:
            type_of_noise = random.randint(0,len(noises)-1)
            
          r,g,b = cv2.split(image)
          r = noisy(noises[type_of_noise],r)
          g = noisy(noises[type_of_noise],g)
          b = noisy(noises[type_of_noise],b)
          image = cv2.merge((r,g,b))
      for j in range(random.randint(5,20)):
        size_square = 20   #The value is to be between 1 and 30
        x = np.random.randint(0,dim - 1)
        y = np.random.randint(0,dim - 1)
        max_height_x = random.randint(0,dim - size_square)
        max_height_y = random.randint(0,dim - size_square)
        r,g,b = cv2.split(image)
        r = random_square(r,x,y,max_height_x,max_height_y)
        x = np.random.randint(0,dim - 1)
        y = np.random.randint(0,dim - 1)
        max_height_x = random.randint(0,dim - size_square)
        max_height_y = random.randint(0,dim - size_square)
        g = random_square(g,x,y,max_height_x,max_height_y)
        x = np.random.randint(0,dim - 1)
        y = np.random.randint(0,dim - 1)
        max_height_x = random.randint(0,dim - size_square)
        max_height_y = random.randint(0,dim - size_square)
        b = random_square(b,x,y,max_height_x,max_height_y)
        image = cv2.merge((r,g,b))  
      attack_data[k]  =   image
      # print(k)
      k = k+1
   
      if  k == 100000  :
        if  FLAG ==0:
          FLAG = 1
          y_pred =model.predict(attack_data)

          attack_data_final,attack_y_final  = normalize_attack(attack_data,y_pred)
          attack_y_max = np.argmax(attack_y_final,axis=-1)
          k= 0
          evalu_quar(attack_y_max)
        else:
          y_pred =model.predict(attack_data)

          tmp_data,tmp_y  = normalize_attack(attack_data,y_pred)
          attack_data_final = np.concatenate((attack_data_final,tmp_data),axis=0)
          k=0
          attack_y_final =np.concatenate((attack_y_final, tmp_y), axis=0)

    if number_attack_vector >100000 :

     return attack_data_final, attack_y_final
    else:
      y_pred =model.predict(attack_data)

      return attack_data,y_pred


In [ ]:

np.random.seed(100)

seta = np.random.randint(0,1000000,size=(10,))

In [ ]:
np.random.seed(100)

seta = np.random.randint(0,1000000,size=(10,))
std = []

for i in tqdm(seta):
  np.random.seed(i)
  attack_data, y_pred = query(1000)
  attack_y_max = np.argmax(y_pred,axis=-1)
  if len(np.unique(attack_y_max))>8:
    k = np.std(attack_y_max)
    std.append(k)

In [ ]:
np.where(std == np.max(std))[0].shape

In [ ]:

np.random.seed(seta[np.where(std == np.max(std))[0][0]])

attack_data,attack_y = query(1000000)

attack_y_max = np.argmax(attack_y,axis=-1)
evalu_quar(attack_y_max)

In [ ]:
np.save("attack_data_10000000.npy",attack_data)

np.save("attack_y_1000000.npy",attack_y)

In [ ]:
attack_data1,attack_y1 = normalize_attack(attack_data,attack_y)


In [ ]:
attack_y_max = np.argmax(attack_y1,axis=-1)
evalu_quar(attack_y_max)

In [ ]:
print(y_pred.shape)
print(attack_data.shape)

In [ ]:
for i in range(10):
    plt.imshow(attack_data[i*35,:,:])
    
    plt.show()

In [ ]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

In [ ]:
base_model=MobileNet(weights='imagenet',input_shape = (32,32,3),include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
# x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(512,activation='relu')(x) #dense layer 2
x=Dense(256,activation='relu')(x) #dense layer 3
preds =Dense(10,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)
for layer in base_model.layers:
	layer.trainable = False
opt = optimizers.Adam(0.001)
model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['acc'])

model.summary()


In [ ]:
attack_y_max  =  keras.utils.to_categorical(attack_y_max,10)
file_path ="cifar_attack.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]
model.fit(attack_data, attack_y_max, epochs=200, verbose=1, callbacks=callbacks_list, validation_data=(x_test,y_test))